# Creating Datasest from Kaggle Dataset

In order to fine tune the trained model in "Image Enhancement" notebook, we decided to do the fine tuning on a dataset from kaggle.  
The chosen dataset was the following dataset:  
https://www.kaggle.com/datasets/sanidhyak/human-face-emotions

Despite all of the efforts done, which could be seen below, we were not able to feed pictures to the pre-trained model because it needed a special kind of input. Although we tried to use the model pre-processing method, we were not able to feed the pictures to the model again.  

Finally, we moved on and we decided to use the .h5 pre-trained model directly as image enhancement method.

### 1- Import Necessary Libraries

In [1]:
import os
import math
import keras
import shutil
import numpy as np
from PIL import Image
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import array_to_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.preprocessing import image_dataset_from_directory

from IPython.display import display

2023-08-05 20:04:05.391303: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 2- Creating low resolution images from Face Dataset

In [2]:
input_dir = "/Users/kavian/Desktop/data/high_resolution_images"
output_dir = "/Users/kavian/Desktop/data/low_resolution_images"
scale_factor = 0.5  # Adjust this to set the desired low-resolution scale factor

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".gif") :
        img = Image.open(os.path.join(input_dir, filename))
        low_res_img = img.resize((int(img.width * scale_factor), int(img.height * scale_factor)), Image.LANCZOS)
        low_res_img.save(os.path.join(output_dir, filename))


### 3- Split Dataset to train, validation and test

In this part we implemented chat gpt to split our dataset to test, train, and validation with hight and low resolution sub folders.

In [3]:
from sklearn.model_selection import train_test_split
import random
import os
import shutil

def create_subdirectories(root_dir, subdirs):
    for subdir in subdirs:
        os.makedirs(os.path.join(root_dir, subdir), exist_ok=True)

def split_dataset(low_resolution_source_dir, high_resolution_source_dir, train_dir, test_dir, validation_dir, split_ratio=(0.8, 0.1, 0.1), random_seed=42):
    # Get the list of low-resolution image filenames and sort them for consistency
    low_resolution_file_list = os.listdir(low_resolution_source_dir)
    low_resolution_file_list.sort()

    # Get the list of high-resolution image filenames and sort them for consistency
    high_resolution_file_list = os.listdir(high_resolution_source_dir)
    high_resolution_file_list.sort()

    # Shuffle both lists with a fixed random seed
    random.seed(random_seed)
    random.shuffle(low_resolution_file_list)
    random.seed(random_seed)
    random.shuffle(high_resolution_file_list)

    # Split the file lists into training, test, and validation sets
    train_files_low_res, test_files_low_res = train_test_split(low_resolution_file_list, test_size=split_ratio[1] + split_ratio[2], random_state=random_seed)
    test_files_low_res, validation_files_low_res = train_test_split(test_files_low_res, test_size=split_ratio[2] / (split_ratio[1] + split_ratio[2]), random_state=random_seed)

    # Create the directories for train, test, and validation
    create_subdirectories(train_dir, ["high_resolution", "low_resolution"])
    create_subdirectories(test_dir, ["high_resolution", "low_resolution"])
    create_subdirectories(validation_dir, ["high_resolution", "low_resolution"])

    # Copy low-resolution images to their respective directories
    for filename in train_files_low_res:
        shutil.copy(os.path.join(low_resolution_source_dir, filename), os.path.join(train_dir, "low_resolution", filename))
    for filename in test_files_low_res:
        shutil.copy(os.path.join(low_resolution_source_dir, filename), os.path.join(test_dir, "low_resolution", filename))
    for filename in validation_files_low_res:
        shutil.copy(os.path.join(low_resolution_source_dir, filename), os.path.join(validation_dir, "low_resolution", filename))

    # Split the file lists for high-resolution images using the same random seed
    train_files_high_res = [filename for filename in high_resolution_file_list if filename in train_files_low_res]
    test_files_high_res = [filename for filename in high_resolution_file_list if filename in test_files_low_res]
    validation_files_high_res = [filename for filename in high_resolution_file_list if filename in validation_files_low_res]

    # Copy high-resolution images to their respective directories
    for filename in train_files_high_res:
        shutil.copy(os.path.join(high_resolution_source_dir, filename), os.path.join(train_dir, "high_resolution", filename))
    for filename in test_files_high_res:
        shutil.copy(os.path.join(high_resolution_source_dir, filename), os.path.join(test_dir, "high_resolution", filename))
    for filename in validation_files_high_res:
        shutil.copy(os.path.join(high_resolution_source_dir, filename), os.path.join(validation_dir, "high_resolution", filename))

# Set the random seed for reproducibility
random_seed = 42

low_resolution_source_dir = "/Users/kavian/Desktop/data/low_resolution_images"
high_resolution_source_dir = "/Users/kavian/Desktop/data/high_resolution_images"
train_dir = "/Users/kavian/Desktop/data/train"
test_dir = "/Users/kavian/Desktop/data/test"
validation_dir = "/Users/kavian/Desktop/data/validation"
split_dataset(low_resolution_source_dir, high_resolution_source_dir, train_dir, test_dir, validation_dir, random_seed=random_seed)


### 4- Fine Tuning

### 4-1- Importing pre-trained model from "Image Enhancement" notebook training

In [4]:
model = tf.keras.models.load_model('/Users/kavian/Desktop/GBC/Second Semester/6- DL II Math/Final Project/DLIIMathProject/Notebooks/Image Enhancement/Image_Enhancement_before_finetuning.h5')

In [5]:
# Freeze all layers except the last few layers
#for layer in pretrained_model.layers[:-2]:
#    layer.trainable = False

In [6]:
print(model.summary())


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 1)]   0         
                                                                 
 conv2d (Conv2D)             (None, None, None, 64)    1664      
                                                                 
 conv2d_1 (Conv2D)           (None, None, None, 64)    36928     
                                                                 
 conv2d_2 (Conv2D)           (None, None, None, 32)    18464     
                                                                 
 conv2d_3 (Conv2D)           (None, None, None, 9)     2601      
                                                                 
 tf.nn.depth_to_space (TFOp  (None, None, None, 1)     0         
 Lambda)                                                         
                                                             

### 4-2- Preparing dataset to feed the model

In [15]:
import tensorflow as tf

def preprocess_input(image):
    target_size = (256, 256)
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image, target_size)
    image = image / 255.0  
    return image

batch_size = 32
num_epochs = 10

train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
validation_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_data_generator.flow_from_directory(
    train_dir,
    batch_size=batch_size,
    class_mode='input',
    shuffle=True
)

validation_generator = validation_data_generator.flow_from_directory(
    validation_dir,
    batch_size=batch_size,
    class_mode='input',
    shuffle=False
)


Found 346 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


In [16]:
# Train the model on the new dataset
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    epochs=num_epochs
)

Epoch 1/10


UnimplementedError: Graph execution error:

Detected at node 'model/conv2d/Relu' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/Users/kavian/Desktop/venv/venv/tensorflow_cpu/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/kavian/Desktop/venv/venv/tensorflow_cpu/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/Users/kavian/Desktop/venv/venv/tensorflow_cpu/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/Users/kavian/Desktop/venv/venv/tensorflow_cpu/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/asyncio/base_events.py", line 604, in

We encountered this error over and over. It seems that it goes back to the number of channels for the feeding image to the model. It should be grayscale.  
We searched for this problem and found this:  
https://stackoverflow.com/questions/61796021/unimplementederror-fused-conv-implementation-does-not-support-grouped-convoluti  
and  
https://stackoverflow.com/questions/73130599/tensorflow-fused-conv-implementation-does-not-support-grouped-convolutions  

But they didn't solve our problem.  
  
We will work on that later.